# CoSim - RTED

In [1]:
import numpy as np

import andes
import ams

import pandas as pd

import json

import cvxpy as cp

In [2]:
ams.__version__

'0.6.4.post55.dev0+gcf11b5a'

In [3]:
andes.config_logger(stream_level=20)

In [4]:
sa = andes.load(andes.get_case('ieee14/ieee14_ace.xlsx'),
                setup=True,
                no_output=True,
                )


Working directory: "/home/jwang/work/ams/dev/demo"
> Loaded config from file "/home/jwang/.andes/andes.rc"
> Loaded generated Python code in "/home/jwang/.andes/pycode".
Parsing input file "/home/jwang/.local/lib/python3.10/site-packages/andes/cases/ieee14/ieee14_ace.xlsx"...
Input file parsed in 0.1835 seconds.
System internal structure set up in 0.0427 seconds.


In [5]:
ams.config_logger(stream_level=10)

In [6]:
sp = ams.load(ams.get_case('ieee14/ieee14_rted.xlsx'),
              setup=True,
              )

Working directory: "/home/jwang/work/ams/dev/demo"
Input format guessed as xlsx.
Parsing input file "/home/jwang/work/ams/ams/cases/ieee14/ieee14_rted.xlsx"...
Input file parsed in 0.0687 seconds.
Adjusting the bus index to start from 0.
System set up in 0.0102 seconds.


In [7]:
sp.RTED.setup()

- Generating symbols for RTED
Set constrs power balance: self.constrs["pb"]=cp.sum(self.routine.pd.v) - cp.sum(self.pg) == 0
Set constrs line limits upper bound: self.constrs["lub"]=self.routine.PTDF1.v @ (self.pg - self.routine.pd1.v) - self.routine.PTDF2.v @ self.routine.pd2.v - self.routine.rate_a.v <= 0
Set constrs line limits lower bound: self.constrs["llb"]=- self.routine.PTDF1.v @ (self.pg - self.routine.pd1.v) + self.routine.PTDF2.v @ self.routine.pd2.v - self.routine.rate_a.v <= 0
RTED model set up in 0.0108 seconds.


True

In [8]:
sp.RTED.pg0

RParam: StaticGen.pg0, v=[0. 0. 0. 0. 0.]

In [ ]:
sa = sp.to_andes(setup=False,
                 addfile=andes.get_case('ieee14/ieee14_wt3.xlsx'),
                 overwrite=True,
                 keep=False,
                 no_output=True,
                 )

sa.add("Toggle", dict(model='SynGen', dev="GENROU_4", t=11.0))

sa.setup()

In [ ]:
sp.ACOPF.run()

In [ ]:
sp.ACOPF.pg

In [ ]:
sp.dyn.send()

In [ ]:
sa.PFlow.run()

In [ ]:
idx_tg = sa.TGOV1.idx.v
vmax0 = sa.TGOV1.get(src='VMAX', attr='v', idx=idx_tg)
sa.TGOV1.set(src='VMAX', attr='v', idx=idx_tg, value=10 * vmax0)
vmin0 = sa.TGOV1.get(src='VMIN', attr='v', idx=idx_tg)
sa.TGOV1.set(src='VMIN', attr='v', idx=idx_tg, value=np.zeros_like(vmin0))

In [ ]:
sa.TDS.config.tf = 10
sa.TDS.run()

In [ ]:
sa.TDS.config.tf = 20
sa.TDS.run()

In [ ]:
sp.dyn.send()

In [ ]:
sp.dyn.receive()